In [109]:
import psycopg
import json
from shapely.geometry import shape
import pandas as pd

In [24]:
def run_query(query, fetch_results = False):
    with psycopg.connect(dbname='taxi_availability', user='myuser', password='mypassword', host='localhost', port='5432') as conn:
        with conn.cursor() as cur:
            cur.execute(query)
            if fetch_results:
                try:
                    results = cur.fetchall()
                    return results
                except:
                    print("Query does not produce any results")

In [93]:
query = """
    CREATE TABLE IF NOT EXISTS taxi_availability (
    timestamp timestamptz PRIMARY KEY,
    geom GEOMETRY(MultiPoint, 4326),
    taxi_count integer
);  
        """

In [94]:
run_query(query, True)

Query does not produce any results


In [95]:
with open("data/raw/taxi_availability_2024_12_31.json", "r") as file:
    taxi_availability_list = json.load(file)

In [102]:
conn = psycopg.connect(dbname='taxi_availability', user='myuser', password='mypassword', host='localhost', port='5432')
cur = conn.cursor()
insert_query = "INSERT INTO taxi_availability (timestamp, geom, taxi_count) VALUES (%s, ST_GeomFromText(%s, 4326), %s) \
                ON CONFLICT (timestamp) DO NOTHING;"
for taxi_availability in taxi_availability_list:
    geometry = shape(taxi_availability["features"][0]["geometry"])
    timestamp = taxi_availability["features"][0]["properties"]["timestamp"]
    taxi_count = taxi_availability["features"][0]["properties"]["taxi_count"]
    cur.execute(insert_query, (timestamp, geometry.wkt, taxi_count))
conn.commit()
cur.close()
conn.close()

In [120]:
conn = psycopg.connect(
    dbname="taxi_availability", user="myuser", password="mypassword", host="localhost", port="5432")

In [121]:
pd.read_sql("select timestamp ,ST_AsText(geom) from taxi_availability", conn)

C:\Users\User\AppData\Local\Temp\ipykernel_26396\3614003795.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql("select timestamp ,ST_AsText(geom) from taxi_availability", conn)


,timestamp,st_astext
0,2024-12-30 16:00:36+00:00,"MULTIPOINT((103.62857 1.30203),(103.64316 1.33..."
1,2024-12-30 16:01:36+00:00,"MULTIPOINT((103.62748 1.3026),(103.64316 1.334..."
2,2024-12-30 16:02:36+00:00,"MULTIPOINT((103.62751 1.30263),(103.64316 1.33..."
3,2024-12-30 16:04:06+00:00,"MULTIPOINT((103.62751 1.30263),(103.64316 1.33..."
4,2024-12-30 16:05:36+00:00,"MULTIPOINT((103.62751 1.30261),(103.64316 1.33..."
...,...,...
2785,2024-12-30 14:51:32+00:00,"MULTIPOINT((103.61943 1.27614),(103.63565 1.33..."
2786,2024-12-30 14:52:32+00:00,"MULTIPOINT((103.62051 1.2755),(103.63672 1.288..."
2787,2024-12-30 14:53:32+00:00,"MULTIPOINT((103.62051 1.2755),(103.63672 1.288..."
2788,2024-12-30 14:54:33+00:00,"MULTIPOINT((103.62239 1.27529),(103.63703 1.29..."


In [122]:
pd.read_sql("select count(*) from taxi_availability", conn)

C:\Users\User\AppData\Local\Temp\ipykernel_26396\3468043148.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql("select count(*) from taxi_availability", conn)


,count
0,2790
